<a href="https://colab.research.google.com/github/Djangoum/HashCode/blob/master/Hash2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/Djangoum/HashCode

Cloning into 'HashCode'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 51 (delta 12), reused 40 (delta 7), pack-reused 0
Unpacking objects: 100% (51/51), done.


In [3]:
!ls
%cd HashCode/2019/

HashCode  sample_data
/content/HashCode/2019


In [0]:
import cv2
import numpy as np
from collections import namedtuple

Image = namedtuple("Image", "id ori tagN tags")
Slide = namedtuple("Slide", "id1 id2 tagN tags")

In [0]:
def getImages(path):
  with open(path) as f:
    lines = f.readlines()
  print(len(lines))
  
  images = []
  lines = lines[1:]
  for i in range(len(lines)):
    images.append(getImage(i, lines[i]))
  return images

def getImage(index, row):
  return Image(index, row[0], row[1], row[2:])

def getSlides(images):
  print(f"totalimages {len(images)}")
  
  horizontalImages = [vimage for vimage in images if vimage.ori == 'H']
  horizontalSlides = [Slide(h.id, "", len(h.tags), h.tags) for h in horizontalImages]
  print(f"himages {len(horizontalSlides)}")
  
  verticalSlides = getBestVerticalPairs2(images)
  print(f"vimages {len(verticalSlides)}")

  slides = verticalSlides + horizontalSlides
  sortedList = sorted(slides, key=lambda x: x.tagN)
  sortedList.reverse()
  return sortedList
  
def getBestPair(image, images):
  bestScore = 999999
  bestImage = ""
  for cmpImage in images:
    tagsShared =  sum([int(cmpTag in image.tags) for cmpTag in cmpImage.tags])
    if tagsShared < bestScore:
      bestScore = tagsShared
      bestImage = cmpImage
  return bestImage

def getBestVerticalPairs(images):
  verticalImages = [vimage for vimage in images if vimage.ori == 'V']
  
  verticalPairs = []
  while len(verticalImages) > 0:
    img = verticalImages.pop()
    bestPair = getBestPair(img, verticalImages)
    verticalImages.remove(bestPair)
    verticalPairs.append([img, bestPair])
    
  return [Slide(pair[0].id, pair[1].id, len(set(pair[0].tags + pair[1].tags)), set(pair[0].tags + pair[1].tags)) for pair in verticalPairs]

def getBestVerticalPairs2(images):
  verticalImages = [vimage for vimage in images if vimage.ori == 'V']
  verticalImages = sorted(verticalImages, key=lambda x: x.tagN)
  
  verticalPairs = []
  while len(verticalImages) > 0:
    img = verticalImages.pop()
    bestPair = getBestPair(img, verticalImages[:100])
    verticalImages.remove(bestPair)
    verticalPairs.append([img, bestPair])
  return [Slide(pair[0].id, pair[1].id, len(set(pair[0].tags + pair[1].tags)), set(pair[0].tags + pair[1].tags)) for pair in verticalPairs]
  
def score_transition(slide1, slide2):
    common = len(list(set(slide1).intersection(slide2)))
    not_in_slide1 = len([x for x in slide1 if x not in slide2])
    not_in_slide2 = len([x for x in slide2 if x not in slide1])
    return min(common, not_in_slide1, not_in_slide2)

def print_output(slideShow):
  print(len(slideShow))
  for slide in slideShow:
    print(f"{slide.id1} {slide.id2}".strip())

In [103]:
file = "b_lovely_landscapes.txt"
images = getImages(file)
slides = getSlides(images)

80001
totalimages 80000
himages 80000
vimages 0


In [104]:
slideShow = []
sl = slides.pop()
slideShow.append(sl)

while len(slides) > 0:
  print("Slides len {0}".format(len(slides)), end="\r")
  if len(slides) >= 10:
    compareBag = slides[:10]
  else:
    compareBag = slides
  
  bestSlide = ""
  bestScore = 0
  for cmp in compareBag:
    score = score_transition(sl.tags, cmp.tags)
    if score > bestScore:
      bestScore = score
      bestSlide = cmp
  
  if bestScore == 0:
    sl = slides.pop()
  else:
    slideShow.append(bestSlide)
    slides.remove(bestSlide)
    sl = bestSlide


  

In [105]:
print_output(slideShow)

1
60427


In [86]:
len(images)

1000

21
3
57
3
